<font color="red"> Remaining to do in Lasso/Linear:

- <s>Outliers to be identified and removed </s>

- <s>Analysis to be done after removing outliers as they have significant influence </s>
- Maybe Better plots required to visualize 

</font>

# Data Science in Engineering
## Semesteraufgabe SoSe 2023
### Aufgabe 1: Glanzoptimierung

| Name              | Matrik No. |
|-------------------|:------------:|
| Renuka Gajralwar  |            |
| Saikiran Joshi    |            |
| Hithesh Karanam   |     478001       |
| Taher Muhammedali |   478012   |

## Import Libraries

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from scipy.stats import zscore
import warnings
from warnings import simplefilter

from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.exceptions import ConvergenceWarning

AttributeError: module 'numpy' has no attribute 'MachAr'

## Import Dataset

In [4]:
#import data

df = pd.read_excel('Aufgabe_1_Gloss_Optimization.xlsx')
print(f"Number of observations in the Gloss Dataset is {df.shape[0]}.")
print(f"Number of features in the Gloss Dataset is {df.shape[1]-1}.")
df.head()

Number of observations in the Gloss Dataset is 58.
Number of features in the Gloss Dataset is 14.


,V13,V15,V16,V18,V19,V21,V28,V29,V30,V31,V32,V33,V34,V39,gloss
0,0.10,HMW 1,0.05,LMW 1,0.05,LMW 6,-0.004681,0.103880,0.000972,-0.005881,0.055534,0.058750,-0.004681,8,6200
1,0.12,HMW 1,0.04,LMW 1,0.04,LMW 6,-0.010345,0.112653,0.001479,0.005503,0.040954,0.056777,-0.010345,8,6400
2,0.12,HMW 1,0.06,LMW 1,0.02,LMW 6,-0.005359,0.125634,0.003815,-0.012599,0.074109,0.015341,-0.005359,8,5800
3,0.14,HMW 1,0.04,LMW 1,0.02,LMW 6,-0.004240,0.147946,-0.016650,0.010005,0.039433,0.018663,-0.004240,6,6200
4,0.28,HMW 1,0.08,LMW 1,0.04,LMW 6,-0.026882,0.284841,-0.009568,-0.019216,0.068883,0.038201,-0.026882,8,8600


In [5]:
# Create an array using np.arange
original_array = np.arange(df.shape[1])

# Values to be removed
values_to_remove = np.array([1, 3, 5]) # String columns

# Remove specified values using set differences
filtered_array = np.setdiff1d(original_array, values_to_remove)

# Define a threshold for outlier detection (e.g., 2 or 3)
threshold = 2.5
outlierlist = []

for i in filtered_array:
    # Create a sample dataset (replace this with your actual data)
    data = df.iloc[:,i]

    # Calculate Z-Scores
    z_scores = zscore(data)
    
    outliers = np.where(np.abs(z_scores) > threshold)[0]
    outlierlist = outlierlist + [item for item in outliers if item not in outlierlist]

values = df.iloc[outliers]
print("Indices of outliers:", outliers)
values

Indices of outliers: [37 38 40]


,V13,V15,V16,V18,V19,V21,V28,V29,V30,V31,V32,V33,V34,V39,gloss
37,0.2,LMW 3,0.2,HMW 4,0.0,NaN,0.007180,0.215330,-0.002961,0.025634,0.201124,0.014473,0.007180,8,11800
38,0.2,LMW 3,0.2,HMW 1,0.0,NaN,0.031322,0.195481,0.008419,0.006735,0.183201,0.006038,0.031322,8,13600
40,0.2,LMW 3,0.2,HMW 6,0.0,NaN,0.199251,-0.002632,-0.001479,-0.001118,0.189497,-0.000656,-0.000749,8,13400


In [6]:
df2 = df.copy()
df2 = df2.drop(outliers).reset_index(drop=True)
print(f"Number of observations in the Gloss Dataset without outliers is {df2.shape[0]}.")
print(f"Number of features in the Gloss Dataset without outliers is {df2.shape[1]-1}.")

Number of observations in the Gloss Dataset without outliers is 55.
Number of features in the Gloss Dataset without outliers is 14.


In [7]:
def encoder_transformer(df):
    
    # Define features and target seperately 
    X = df.drop('gloss',axis=1) # Here X is our list of features
    y = df['gloss'] # Here y is our target, gloss

    # Finding column index where features are non-numeric
    string_col_indices = [i for i,col in enumerate (X.columns) if X[col].dtype=='object']

    # Using OneHotEncoder & ColumnTransformer to transform string indices
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'),string_col_indices )], remainder='passthrough')
    X = np.array(ct.fit_transform(X)).astype('float64')
    print(f"After encoder & column transformation, the original dataframe has changed from {df.shape} to an array of {X.shape}")
    
    # Creating a list with all feature names after column transformation
    name_list = ct.get_feature_names_out().tolist()
    name_list.insert(0, "const") # Manually adding a string of "const" to list of names
    
    X = sm.add_constant(X)
    X_df = pd.DataFrame(X, columns=name_list)
    
    return X_df, y

In [8]:
X_all_df, y_all = encoder_transformer(df)

ols_reg = sm.OLS(y_all, X_all_df).fit()

y_pred_all = ols_reg.predict(X_all_df)
rmse_all = mean_squared_error(y_all, y_pred_all, squared=False)
r2_all = r2_score(y_all, y_pred_all)
print("\nRMSE:", rmse_all)
print("R2:", r2_all)

After encoder & column transformation, the original dataframe has changed from (58, 15) to an array of (58, 35)


AttributeError: 'ColumnTransformer' object has no attribute 'get_feature_names_out'

In [ ]:
X_ohne_df, y_ohne = encoder_transformer(df2)

ols_reg2 = sm.OLS(y_ohne, X_ohne_df).fit()

y_pred_ohne = ols_reg2.predict(X_ohne_df)
rmse_ohne = mean_squared_error(y_ohne, y_pred_ohne, squared=False)
r2_ohne = r2_score(y_ohne, y_pred_ohne)
print("\nRMSE:", rmse_ohne)
print("R2:", r2_ohne)

In [ ]:
print(ols_reg2.summary())

In [ ]:
X_all, y_all, name_list_all = encoder_transformer(df)
X_all = sm.add_constant(X_all) # Adding constant
X_all_df = pd.DataFrame(X_all, columns=name_list_all) # Creating a dataframe of the transformed array with the feature name list combined

ols_reg = sm.OLS(y_all, X_all_df).fit()

y_pred_all = ols_reg.predict(X_all_df)
rmse_all = mean_squared_error(y_all, y_pred_all, squared=False)
r2_all = r2_score(y_all, y_pred_all)
print("\nRMSE:", rmse_all)
print("R2:", r2_all)

Is the pair plot required?

Don't think so...

We can remove it no problem -Hithesh

In [ ]:
# # checking pair plot of the dataset
# sns.pairplot(df, kind='reg')
# plt.suptitle('Pair plot for Gloss Optimization dataset', fontsize=16, y=1)

## Correlation Analysis

In [ ]:
#Here we will perform two correlation analysis, Pearson & Spearman

# removing FutureWarning generated from correlation formulas
warnings.simplefilter(action='ignore', category=FutureWarning)

#Checking the relationship between the variables of the given dataset

# Finding the Pearson correlation of the dataset
df_pears = df.corr(method='pearson')

# Finding the Spearman correlation of the dataset
df_spear = df.corr(method='spearman')

In [ ]:
#Plotting both Correlation analysis to visualize and analyse correlation within the gloss dataset

plt.figure(figsize=(20, 10))

# Subplot for Pearson correlation
#Pearson correlation gives the linear realtionship between the variables
plt.subplot(1, 2, 1)
sns.heatmap(df_pears, annot=True, cmap='coolwarm')
plt.title('Pearson Correlation', fontsize=18, y=1)


# Subplot for Spearman correlation
#Spearman correlation gives the monotonous realtionship (non-linear) between the variables
plt.subplot(1, 2, 2)
sns.heatmap(df_spear, annot=True, cmap='coolwarm')
plt.title('Spearman Correlation', fontsize=18, y=1)

plt.tight_layout()
plt.show()

<b>Pearson Correlation Observation</b>
- Here we can observe that features <i>V32 and V39</i> are showing medium-strong correlation to our target <i>gloss</i>, <i>0.42 & 0.55</i> respectively.
- Subsequently, we can also observe that both <i>V32 and V39</i> are strongly correlated as per this correlation method with a strong correlation value of 0.73

<b>Spearman Correlation Observation</b>
- Spearman also reinforces the observations above from Pearson, for features <i>V32 and V39</i>.
- Additionally, we can observe a medium correlation of feature <i>V19</i> with our target <i>gloss</i>

Note - Non-numeric features such as <i>V15, V18 and V21</i> are not considered in the above correlation analysis

## Transforming Categorical Variables

In [ ]:
# Define features and target seperately 
X = df.drop('gloss',axis=1) # Here X is our list of features
y = df['gloss'] # Here y is our target, gloss

# Finding column index where features are non-numeric
string_col_indices = [i for i,col in enumerate (X.columns) if X[col].dtype=='object']

# Using OneHotEncoder & ColumnTransformer to transform string indices
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'),string_col_indices )], remainder='passthrough')
X = np.array(ct.fit_transform(X)).astype('float64')
print(f"After encoder & column transformation, the original dataframe has changed from {df.shape} to an array of {X.shape}")

# Creating a list with all feature names after column transformation
name_list = ct.get_feature_names_out().tolist()
name_list.insert(0, "const") # Manually adding a string of "const" to list of names

## OLS Regression
(of the full dataset)

In [ ]:
# OLS Regression Analysis of the dataset
X_sm = sm.add_constant(X) # Adding constant
X_sm_df = pd.DataFrame(X_sm, columns=name_list) # Creating a dataframe of the transformed array with the feature name list combined
ols_reg = sm.OLS(y, X_sm_df).fit()
print(ols_reg.summary())

Not sure which one of the below two is correct, ohne constant or with constant

In [ ]:
max_p = ols_reg.summary2().tables[1]['P>|t|'].max()

while max_p > 0.05:
    max_index = ols_reg.summary2().tables[1]['P>|t|'][ols_reg.summary2().tables[1]['P>|t|'] == max_p].index[0]
    X_sm_df.drop(columns=max_index, inplace=True)
    ols_reg = sm.OLS(y, X_sm_df).fit()
    max_p = ols_reg.summary2().tables[1]['P>|t|'].max()

print(ols_reg.summary())

In [ ]:
max_p = ols_reg.summary2().tables[1][1:]['P>|t|'].max()

while max_p > 0.05:
    max_index = ols_reg.summary2().tables[1][1:]['P>|t|'][ols_reg.summary2().tables[1][1:]['P>|t|'] == max_p].index[0]
    X_sm_df.drop(columns=max_index, inplace=True)
    ols_reg = sm.OLS(y, X_sm_df).fit()
    max_p = ols_reg.summary2().tables[1][1:]['P>|t|'].max()

print(ols_reg.summary())

### Splitting the dataset

In [ ]:
# Since we only have 58 observations, it would be better to train the model on the basis of a siginificant training split of dataset for accurate predictions
# Here we are using sklearn train_test_split to split the data using a pre-assigned random state in order to get consistent results at every run

# Split the data into 90% training and 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### Scalar transformation of X

In [ ]:
# define scaler object
sc = StandardScaler()
# fit scaler based on training data, scale training data
X_train_sc = sc.fit_transform(X_train)
# scale test data
X_test_sc = sc.transform(X_test)

## Linear Regression

In [ ]:
# Define the regressor and perform the fit based on the scaled training data
linear_reg = LinearRegression()
linear_reg.fit(X_train_sc, y_train)

# Predict based on the features for both testing and training data
y_pred_train_linear = linear_reg.predict(X_train_sc) # Predicting y for training data. This is expected to have a good fit as the model is trained using this dataset.
y_pred_test_linear = linear_reg.predict(X_test_sc)  # predicting y for testing data

# Print the metrics R2 and RMSE for both training and testing data
print(f'Train metrics: R2: {r2_score(y_train, y_pred_train_linear)}, RMSE: {mean_squared_error(y_train, y_pred_train_linear, squared=False)}')
print(f'Test metrics: R2: {r2_score(y_test, y_pred_test_linear)}, RMSE: {mean_squared_error(y_test, y_pred_test_linear, squared=False)}')

In [ ]:
# Calculate residuals for OLS regression
linear_train_residuals = y_train - y_pred_train_linear
linear_test_residuals = y_test - y_pred_test_linear

# Define threshold for identifying outliers
outlier_threshold = 2.5 # The outlier threshold can be tweaked as per requirements

# Create Residuals vs. Predicted Values plot for HuberRegressor with marked outliers
plt.figure(figsize=(10, 5))

# Create Residuals vs. Predicted Values plot for OLS regression with marked outliers
plt.subplot(1, 2, 1)
plt.scatter(y_pred_train_linear, linear_train_residuals, label='Train')
plt.scatter(y_pred_test_linear, linear_test_residuals, label='Test')

# Mark outliers for OLS regression using outlier threshold
ols_train_outliers = np.abs(linear_train_residuals) > outlier_threshold * np.std(linear_train_residuals)
ols_test_outliers = np.abs(linear_test_residuals) > outlier_threshold * np.std(linear_test_residuals)
plt.scatter(y_pred_train_linear[ols_train_outliers], linear_train_residuals[ols_train_outliers],
            color='r', label='Train Outliers', marker='s')
plt.scatter(y_pred_test_linear[ols_test_outliers], linear_test_residuals[ols_test_outliers],
            color='r', label='Test Outliers')

plt.axhline(0, color='r', linestyle='--') 
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('OLS Regression Residuals vs. Predicted Values')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#Residuals Plot for Linear Regression
plt.figure(figsize=(8, 5))
sns.residplot(x=y_test, y=y_pred_test_linear, label='Testing Data', scatter_kws={"marker": "*", "color": "red"}) #Plotting testing data
sns.residplot(x=y_train, y=y_pred_train_linear, lowess=True, line_kws={"color":"g"}, label='Training Data', scatter_kws={"marker": "o", "color": "blue"}) #Plotting training data using lowess
plt.legend()
plt.xlabel('Predicted Dissolution')
plt.ylabel('Residuals')
plt.title("Linear Regression Residuals Plot", fontsize=18)
plt.show()

<b>Linear Regression Observation</b>

- As expected, predicted y values based on training data shows a healthy fit with R2 ≈ 1
- However, the prediction of y values based on test data shows a poor fit with R2 ≈ 0.5
- The RMSE for testing data being significantly higher than that of training data also shows that errors are larger for testing data

From this we can possibly conclude that Linear Regression has not turned out to be good model for our dataset.

## Lasso Regression

In [ ]:
#Suppressing Convergence warnings
simplefilter("ignore", category=ConvergenceWarning)

#Defining alpha range for Lasso regression
alphas = np.logspace(-5, 100, 1000)  
lassocv_reg = LassoCV(alphas=alphas, cv=5) #cross validation = 5

# Fitting the lasso regressor model based on training data
lassocv_reg.fit(X_train_sc, y_train)

# Predicting y values based on training and testing data
y_pred_test_lasso = lassocv_reg.predict(X_test_sc)
y_pred_train_lasso = lassocv_reg.predict(X_train_sc)

# Calculate RMSE with the best alpha
print(f'Train metrics: R2: {r2_score(y_train, y_pred_train_lasso)}, RMSE: {mean_squared_error(y_train, y_pred_train_lasso, squared=False)}')
print(f'Test metrics: R2: {r2_score(y_test, y_pred_test_lasso)}, RMSE: {mean_squared_error(y_test, y_pred_test_lasso, squared=False)} \n')

# Get the best alpha value
best_alpha = lassocv_reg.alpha_
print(f"Best Alpha: {best_alpha}")

In [ ]:
#Residuals Plot for Lasso Regression
plt.figure(figsize=(8, 5))
sns.residplot(x=y_test, y=y_pred_test_lasso, label='Testing Data', scatter_kws={"marker": "*", "color": "red"}) #Plotting testing data
sns.residplot(x=y_train, y=y_pred_train_lasso, lowess=True, line_kws={"color":"g"}, label='Training Data', scatter_kws={"marker": "o", "color": "blue"}) #Plotting training data using lowess
plt.legend()
plt.xlabel('Predicted Dissolution')
plt.ylabel('Residuals')
plt.title("Lasso Regression Residuals Plot", fontsize=18)
plt.show()

<b>Lasso Regression Observation</b>

- As expected, again the predicted y values based on training data shows a healthy fit with R2 ≈ 1 for Lasso regression model
- The prediction of y values based on test data shows a poor fit with R2 ≈ 0.6. However, this is still a better fit in comparison to Linear Regression model.
- The RMSE for testing is also in Lasso's case significantly higher than that of training data
- Linear Regressor performed slightly better than Lasso on training data with better R2 and lower RMSE values, however Lasso had a slightly better prediction for training dataset

From this we can possibly conclude that Lasso Regression is also not a good model for our dataset.